In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression


In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))

5


In [3]:
models

[['model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_test'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_features=

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4023
4023
4023


In [9]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [10]:
test_model_name = models[0][0]
test_model = eval(models[1][0])
test_model_features = eval(models[2][0])
test_model_ev = eval(models[3][0])

In [11]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [12]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena']

In [13]:
test_model_features

['B_win_by_Decision_Majority',
 'B_win_by_TKO_Doctor_Stoppage',
 'R_win_by_Decision_Majority',
 'R_odds',
 'R_current_lose_streak',
 'R_Reach_cms',
 'B_draw',
 'total_round_dif',
 'R_ev',
 'B_ev']

In [14]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, True))

    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train)

    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
        
    

Current best score is: 24.45639412549921
NO IMPROVEMENT
FINAL BEST SCORE: 24.45639412549921


Starting New Run for LogisticRegression


Previous Best Score: 24.45639412549921
solver: newton-cg penalty: l2 tol: 5.993308915320019 Score: 24.45639412549921
solver: newton-cg penalty: l2 tol: 7.191970698384022 Score: 24.45639412549921
solver: newton-cg penalty: l2 tol: 4.794647132256015 Score: 24.45639412549921
solver: newton-cg penalty: l2 tol: 7.095412513261494 Score: 24.45639412549921
solver: newton-cg penalty: none tol: 5.993308915320019 Score: 24.45639412549921
solver: newton-cg penalty: none tol: 7.191970698384022 Score: 24.45639412549921
solver: newton-cg penalty: none tol: 4.794647132256015 Score: 24.45639412549921
solver: newton-cg penalty: none tol: 7.095412513261494 Score: 24.45639412549921
solver: lbfgs penalty: l2 tol: 5.993308915320019 Score: 24.45639412549921
solver: lbfgs penalty: l2 tol: 7.191970698384022 Score: 24.45639412549921
solver: lbfgs penalty: l2 tol: 4.794647132256

In [11]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(4023, 10)

(4023,)

(4023, 2)

(200, 10)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-600.0,450.0,0.864661,0.135339,0
1,120.0,-140.0,0.454314,0.545686,1
2,-115.0,-105.0,0.504325,0.495675,0
3,-135.0,115.0,0.549642,0.450358,0
4,400.0,-500.0,0.169111,0.830889,1
...,...,...,...,...,...
195,-550.0,425.0,0.857653,0.142347,0
196,-105.0,-115.0,0.499599,0.500401,0
197,-460.0,365.0,0.806373,0.193627,0
198,-920.0,610.0,0.926645,0.073355,0



          Number of matches: 200
          Number of bets: 91
          Number of winning bets: 53
          Number of losing bets: 38
          Number of underdog bets: 66
          Number of underdog wins: 33
          Number of underdog losses: 33
          Number of Favorite bets: 24
          Number of favorite wins: 19
          Number of favorite losses: 5
          Number of even bets: 1
          Number of even wins: 1
          Number of even losses: 0
          Profit: 1323.8875861704942
          Profit per bet: 14.548215232642793
          Profit per match: 6.619437930852471
          
          


In [12]:
score

1323.8875861704942

In [15]:
test_model_ev

0

In [16]:
print(test_model_name) 
print(models[0][1])
print()
print(test_model)
print(eval(models[1][1]))
print()
print(test_model_features) 
print(eval(models[2][1]))
print()
print(test_model_ev)
print(eval(models[3][1]))

model_1
model_1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=75, solver='lbfgs', tol=5.993308915320019,
                   verbose=0, warm_start=False)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=75, solver='lbfgs', tol=5.993308915320019,
                   verbose=0, warm_start=False)

['B_win_by_Decision_Majority', 'B_win_by_TKO_Doctor_Stoppage', 'R_win_by_Decision_Majority', 'R_odds', 'R_current_lose_streak', 'R_Reach_cms', 'B_draw', 'total_round_dif', 'R_ev', 'B_ev']
['B_win_by_Decision_Majority', 'B_win_by_TKO_Doctor_Stoppage', 'R_win_by_Decision_Majority', 'R_odds', 'R_current_lose_s

In [17]:
#models[0][0] =  test_model_name 
#models[1][0] = test_model
#models[2][0] = test_model_features
#models[3][0] = test_model_ev

In [18]:
models

[['model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_test'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_features=

In [19]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [20]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [21]:
data

[['model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_test'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_features=